In [1]:
import os
import sys
sys.path.append('..')

import json
import numpy as np
from scipy.spatial import ConvexHull, Delaunay, cKDTree
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from ipywidgets import interact, FloatSlider

from CalixVolApp.utils.paths import get_project_path
from CalixVolApp.calculation.visualization import *
from CalixVolApp.calculation.calculation import *

In [2]:
def read_coordinates(filename):
    atoms = []
    coords = []
    with open(filename, 'r') as file:
        for line in file:
            parts = line.strip().split()
            atoms.append(parts[0])
            coords.append([float(x) for x in parts[1:4]])
    return atoms, np.array(coords)

In [3]:
def estimate_internal_volume(filename, grid_resolution=0.1):
    atoms, coords = read_coordinates(filename)
    atom_radii = np.array([vdw_radii.get(atom, 1.5) for atom in atoms])

    hull = ConvexHull(coords)
    hull_points = coords[hull.vertices]

    max_radius = max(vdw_radii.values())
    min_coords = np.min(hull_points, axis=0) - max_radius
    max_coords = np.max(hull_points, axis=0) + max_radius
    
    x = np.arange(min_coords[0], max_coords[0], grid_resolution)
    y = np.arange(min_coords[1], max_coords[1], grid_resolution)
    z = np.arange(min_coords[2], max_coords[2], grid_resolution)
    X, Y, Z = np.meshgrid(x, y, z)
    grid_points = np.vstack((X.ravel(), Y.ravel(), Z.ravel())).T

    delaunay = Delaunay(hull_points)
    inside_hull = delaunay.find_simplex(grid_points) >= 0
    points_in_hull = grid_points[inside_hull]

    tree = cKDTree(coords)
    max_atom_radius = max(atom_radii)
    indices = tree.query_ball_point(points_in_hull, r=max_atom_radius)
    inside_atom = np.zeros(len(points_in_hull), dtype=bool)

    for i, inds in enumerate(indices):
        point = points_in_hull[i]
        for j in inds:
            distance = np.linalg.norm(point - coords[j])
            if distance <= atom_radii[j]:
                inside_atom[i] = True
                break

    volume_total = hull.volume
    volume_atom = np.sum(inside_atom) * (grid_resolution ** 3)
    volume_cavity = np.sum(~inside_atom) * (grid_resolution ** 3)

    return (volume_total, volume_atom, volume_cavity)

### Цвета атомов и их Ван-дер-Ваальсовы радиусы

In [4]:
with open(os.path.join(get_project_path(), 'CalixVolApp', 'data', 'vdw', 'vdw_colors.json'), "r") as file:
    atom_colors = json.load(file)

with open(os.path.join(get_project_path(), 'CalixVolApp', 'data', 'vdw', 'vdw_radius.json'), "r") as file:
    vdw_radii = json.load(file)

# РЕЗУЛЬТАТ РАСЧЕТА

In [5]:
filename = os.path.join(get_project_path(), 'CalixVolApp', 'data', 'molecules', 'correct 1 calix.txt')

volumes = estimate_internal_volume(filename)

atoms, coords = read_coordinates(filename)

volume_total, volume_atom, volume_cavity = volumes
print(f"Общий объем выпуклой оболочки: {volume_total:.2f} Å³")
print(f"Объем, занятый атомами: {volume_atom:.2f} Å³")
print(f"Объем полости: {volume_cavity:.2f} Å³")


Общий объем выпуклой оболочки: 272.16 Å³
Объем, занятый атомами: 155.65 Å³
Объем полости: 116.50 Å³


# ШАГ 1. Визуализация атомов без ВВР

In [41]:
def plot_simple_3d_molecule(azim=45, elev=30):
    fig = plt.figure(figsize=(16, 10))
    ax = fig.add_subplot(111, projection='3d', facecolor='whitesmoke')

    # Отображение атомов с эффектом прозрачности и увеличением размера точек
    for atom, coord in zip(atoms, coords):
        color = atom_colors.get(atom, 'grey')  # Серый цвет по умолчанию
        ax.scatter(coord[0], coord[1], coord[2], color=color, s=400, edgecolors='k', alpha=0.9)

    # Настройка осей
    ax.set_xlabel('X координата', fontsize=12, fontweight='bold', labelpad=15)
    ax.set_ylabel('Y координата', fontsize=12, fontweight='bold', labelpad=15)
    ax.set_zlabel('Z координата', fontsize=12, fontweight='bold', labelpad=15)
    ax.set_title('3D визуализация молекулы', fontsize=18, fontweight='bold', pad=30)

    # Установка углов обзора
    ax.view_init(elev=elev, azim=azim)

    # Удаление дубликатов элементов для легенды
    unique_atoms = set(atoms)
    legend_elements = []
    for element in unique_atoms:
        color = atom_colors.get(element, 'grey')
        legend_elements.append(Line2D([0], [0], marker='o', color='w', label=element,
                                      markerfacecolor=color, markersize=14, markeredgecolor='k'))

    # Добавление легенды
    ax.legend(handles=legend_elements, loc='upper right', fontsize=12, frameon=True, facecolor='white', edgecolor='black')

    # Отключение видимости осевых рамок
    ax.grid(False)
    ax.xaxis.pane.set_edgecolor('w')
    ax.yaxis.pane.set_edgecolor('w')
    ax.zaxis.pane.set_edgecolor('w')

    ax.set_axis_off()

    # plt.savefig('step_1.png', format='png', dpi=300, bbox_inches='tight', pad_inches=0.1)

    # Показ графика
    plt.tight_layout()
    plt.show()


# Интерактивные слайдеры для управления углами
interact(plot_simple_3d_molecule,
         azim=FloatSlider(min=0, max=360, step=1, value=360, description='Азимут:'),
         elev=FloatSlider(min=-90, max=90, step=1, value=-64, description='Угол высоты:'));


interactive(children=(FloatSlider(value=360.0, description='Азимут:', max=360.0, step=1.0), FloatSlider(value=…

# ШАГ 2. Визуализация вандерваальсовых радиусов

In [42]:
def plot_molecule(azim=45, elev=30):
    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d', facecolor='whitesmoke')

    # Отображение атомов в виде сфер с учетом ван-дер-ваальсовых радиусов
    for atom, coord in zip(atoms, coords):
        color = atom_colors.get(atom, '#808080')  # Серый цвет по умолчанию
        radius = vdw_radii.get(atom, 1.5)  # Используем 1.5 Å по умолчанию
        # Создаем сферу
        u, v = np.mgrid[0:2*np.pi:30j, 0:np.pi:15j]
        x = radius * np.cos(u) * np.sin(v) + coord[0]
        y = radius * np.sin(u) * np.sin(v) + coord[1]
        z = radius * np.cos(v) + coord[2]
        ax.plot_surface(x, y, z, color=color, linewidth=2, antialiased=True, shade=True, alpha=0.8)

    # Добавление связей между атомами
    for i, coord1 in enumerate(coords):
        for j, coord2 in enumerate(coords):
            if i < j:
                distance = np.linalg.norm(coord1 - coord2)
                if distance < 2.0:  # Условие для отображения связи
                    ax.plot([coord1[0], coord2[0]], 
                            [coord1[1], coord2[1]], 
                            [coord1[2], coord2[2]], 
                            color='grey', linewidth=1.5, alpha=0.7)

    # Настройка осей
    ax.set_title('3D визуализация молекулы с ван-дер-ваальсовыми радиусами', fontsize=18, fontweight='bold', pad=30)
    ax.dist = 12

    # Установка углов обзора
    ax.view_init(elev=elev, azim=azim)

    # Добавление легенды
    unique_atoms = set(atoms)
    legend_elements = []
    for element in unique_atoms:
        color = atom_colors.get(element, '#808080')
        legend_elements.append(Line2D([0], [0], marker='o', color='w', label=element,
                                      markerfacecolor=color, markersize=14, markeredgecolor='k'))
    ax.legend(handles=legend_elements, loc='upper right', fontsize=12, frameon=True, facecolor='white', edgecolor='black')

    # Отключение видимости осевых рамок
    ax.grid(False)
    ax.set_axis_off()

    # plt.savefig('step_2.png', format='png', dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.tight_layout()
    plt.show()

# Виджеты для управления углами
interact(plot_molecule, 
         azim=FloatSlider(min=0, max=360, step=1, value=360, description='Азимут:'),
         elev=FloatSlider(min=-90, max=90, step=1, value=-64, description='Угол высоты:'));

interactive(children=(FloatSlider(value=360.0, description='Азимут:', max=360.0, step=1.0), FloatSlider(value=…

# ШАГ 2. Конвекс холл

In [43]:
def plot_molecule_with_hull(azim=45, elev=30):
    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d', facecolor='whitesmoke')

    # Отображение атомов в виде сфер с учетом ван-дер-ваальсовых радиусов
    for atom, coord in zip(atoms, coords):
        color = atom_colors.get(atom, '#808080')  # Серый цвет по умолчанию
        radius = vdw_radii.get(atom, 1.5)  # Используем 1.5 Å по умолчанию
        # Создаем сферу
        u, v = np.mgrid[0:2*np.pi:30j, 0:np.pi:15j]
        x = radius * np.cos(u) * np.sin(v) + coord[0]
        y = radius * np.sin(u) * np.sin(v) + coord[1]
        z = radius * np.cos(v) + coord[2]
        ax.plot_surface(x, y, z, color=color, linewidth=0, antialiased=True, shade=True, alpha=0.6)

    # Отображение выпуклой оболочки
    hull = ConvexHull(coords)
    for simplex in hull.simplices:
        triangle = coords[simplex]
        x = triangle[:, 0]
        y = triangle[:, 1]
        z = triangle[:, 2]
        # Создаем массивы для замкнутого треугольника
        x = np.append(x, x[0])
        y = np.append(y, y[0])
        z = np.append(z, z[0])
        ax.plot_trisurf(x, y, z, color='cyan', alpha=0.5, linewidth=0)

    # Настройка осей с увеличенным размером шрифта и жирными линиями
    ax.set_xlabel('X координата', fontsize=14, fontweight='bold', labelpad=15)
    ax.set_ylabel('Y координата', fontsize=14, fontweight='bold', labelpad=15)
    ax.set_zlabel('Z координата', fontsize=14, fontweight='bold', labelpad=15)
    ax.set_title('3D визуализация молекулы с выпуклой оболочкой', fontsize=18, fontweight='bold', pad=30)

    # Настройка лимитов осей для лучшей центровки молекулы и отдаления
    max_radius = max(vdw_radii.values())
    min_coords = np.min(coords, axis=0) - max_radius
    max_coords = np.max(coords, axis=0) + max_radius
    ax.set_xlim([min_coords[0] - 2, max_coords[0] + 2])
    ax.set_ylim([min_coords[1] - 2, max_coords[1] + 2])
    ax.set_zlim([min_coords[2] - 2, max_coords[2] + 2])

    # Настройка расстояния до осей для лучшего обзора
    ax.dist = 12

    # Настройка углов обзора
    ax.view_init(elev=elev, azim=azim)

    # Добавление легкого градиента фона для лучшей визуализации
    ax.xaxis.set_pane_color((0.9, 0.9, 0.9, 0.5))
    ax.yaxis.set_pane_color((0.9, 0.9, 0.9, 0.5))
    ax.zaxis.set_pane_color((0.9, 0.9, 0.9, 0.5))

    # Удаляем дубликаты элементов для легенды
    unique_atoms = set(atoms)
    legend_elements = []
    for element in unique_atoms:
        color = atom_colors.get(element, '#808080')
        legend_elements.append(Line2D([0], [0], marker='o', color='w', label=element,
                                      markerfacecolor=color, markersize=14, markeredgecolor='k'))

    # Добавление легенды с увеличенным шрифтом и рамкой
    ax.legend(handles=legend_elements, loc='upper right', fontsize=12, frameon=True, facecolor='white', edgecolor='black')

    # Отключение видимости осевых рамок для лучшего визуального восприятия
    ax.grid(False)
    ax.xaxis.pane.set_edgecolor('w')
    ax.yaxis.pane.set_edgecolor('w')
    ax.zaxis.pane.set_edgecolor('w')

    ax.set_axis_off()

    # plt.savefig('step_3.png', format='png', dpi=300, bbox_inches='tight', pad_inches=0.1)

    # Показ графика
    plt.tight_layout()
    plt.show()

# Создаем виджеты для изменения углов
interact(plot_molecule_with_hull, 
         azim=FloatSlider(min=0, max=360, step=1, value=360, description='Азимут:'),
         elev=FloatSlider(min=-90, max=90, step=1, value=-64, description='Угол высоты:'));

interactive(children=(FloatSlider(value=360.0, description='Азимут:', max=360.0, step=1.0), FloatSlider(value=…

# ШАГ 3. Создание 3D сетки точек, покрывающую молекулу

In [44]:
grid_resolution = 0.1

# Вычисление выпуклой оболочки
hull = ConvexHull(coords)

# Определение границ сетки
max_radius = max(vdw_radii.values())
min_coords = np.min(coords[hull.vertices], axis=0) - max_radius
max_coords = np.max(coords[hull.vertices], axis=0) + max_radius

# Создание сетки точек
x = np.arange(min_coords[0], max_coords[0])
y = np.arange(min_coords[1], max_coords[1])
z = np.arange(min_coords[2], max_coords[2])
X, Y, Z = np.meshgrid(x, y, z)
grid_points = np.vstack((X.ravel(), Y.ravel(), Z.ravel())).T

def plot_grid_and_hull(azim=45, elev=30):
    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d', facecolor='whitesmoke')

    # Отображение сетки точек
    ax.scatter(grid_points[:, 0], grid_points[:, 1], grid_points[:, 2],
               color='blue', s=5, alpha=0.6, label='Сетка точек')

    # Отображение атомов в виде сфер с учетом ван-дер-ваальсовых радиусов
    for atom, coord in zip(atoms, coords):
        color = atom_colors.get(atom, '#808080')  # Серый цвет по умолчанию
        radius = vdw_radii.get(atom, 1.5)  # Используем 1.5 Å по умолчанию
        # Создаем сферу
        u, v = np.mgrid[0:2*np.pi:30j, 0:np.pi:15j]
        x = radius * np.cos(u) * np.sin(v) + coord[0]
        y = radius * np.sin(u) * np.sin(v) + coord[1]
        z = radius * np.cos(v) + coord[2]
        ax.plot_surface(x, y, z, color=color, linewidth=0, antialiased=True, shade=True, alpha=0.6)

    # Отображение выпуклой оболочки
    for simplex in hull.simplices:
        triangle = coords[simplex]
        x_triangle = triangle[:, 0]
        y_triangle = triangle[:, 1]
        z_triangle = triangle[:, 2]
        ax.plot_trisurf(x_triangle, y_triangle, z_triangle, color='cyan', alpha=0.5, linewidth=0)

    # Настройка осей
    ax.set_xlabel('X координата', fontsize=14, fontweight='bold', labelpad=15)
    ax.set_ylabel('Y координата', fontsize=14, fontweight='bold', labelpad=15)
    ax.set_zlabel('Z координата', fontsize=14, fontweight='bold', labelpad=15)
    ax.set_title('3D визуализация молекулы с сеткой точек и выпуклой оболочкой', fontsize=18, fontweight='bold', pad=30)

    # Настройка лимитов осей
    ax.set_xlim([min_coords[0] - 2, max_coords[0] + 2])
    ax.set_ylim([min_coords[1] - 2, max_coords[1] + 2])
    ax.set_zlim([min_coords[2] - 2, max_coords[2] + 2])

    # Установка углов обзора
    ax.view_init(elev=elev, azim=azim)

    # Настройка градиента фона
    ax.xaxis.set_pane_color((0.9, 0.9, 0.9, 0.5))
    ax.yaxis.set_pane_color((0.9, 0.9, 0.9, 0.5))
    ax.zaxis.set_pane_color((0.9, 0.9, 0.9, 0.5))

    # Удаление дубликатов в легенде
    unique_atoms = set(atoms)
    legend_elements = []
    for element in unique_atoms:
        color = atom_colors.get(element, '#808080')
        legend_elements.append(Line2D([0], [0], marker='o', color='w', label=element,
                                      markerfacecolor=color, markersize=14, markeredgecolor='k'))
    legend_elements.append(Line2D([0], [0], marker='o', color='w', label='Grid of points',
                                  markerfacecolor='blue', markersize=10, markeredgecolor='k'))

    # Добавление легенды
    ax.legend(handles=legend_elements, loc='upper right', fontsize=12, frameon=True, facecolor='white', edgecolor='black')

    # Отключение видимости осевых рамок
    ax.grid(False)
    ax.xaxis.pane.set_edgecolor('w')
    ax.yaxis.pane.set_edgecolor('w')
    ax.zaxis.pane.set_edgecolor('w')
    ax.set_axis_off()

    # plt.savefig('step_4.png', format='png', dpi=300, bbox_inches='tight', pad_inches=0.1)

    # Показ графика
    plt.tight_layout()
    plt.show()

# Интерактивные слайдеры для управления углами
interact(plot_grid_and_hull,
         azim=FloatSlider(min=0, max=360, step=1, value=360, description='Азимут:'),
         elev=FloatSlider(min=-90, max=90, step=1, value=-64, description='Угол высоты:'));

interactive(children=(FloatSlider(value=360.0, description='Азимут:', max=360.0, step=1.0), FloatSlider(value=…

# ШАГ 4. Определение точек внутри выпуклой оболочки c помощью триангуляции Деллоне

In [45]:
grid_resolution = 0.25
atom_radii = np.array([vdw_radii.get(atom, 1.5) for atom in atoms])

# Вычисление выпуклой оболочки
hull = ConvexHull(coords)
hull_points = coords[hull.vertices]

# Определение границ сетки
max_radius = max(vdw_radii.values())
min_coords = np.min(hull_points, axis=0) - max_radius
max_coords = np.max(hull_points, axis=0) + max_radius

# Создание сетки точек
x = np.arange(min_coords[0], max_coords[0], grid_resolution)
y = np.arange(min_coords[1], max_coords[1], grid_resolution)
z = np.arange(min_coords[2], max_coords[2], grid_resolution)
X, Y, Z = np.meshgrid(x, y, z)
grid_points = np.vstack((X.ravel(), Y.ravel(), Z.ravel())).T

# Определение точек внутри выпуклой оболочки
delaunay = Delaunay(hull_points)
inside_hull = delaunay.find_simplex(grid_points) >= 0
points_in_hull = grid_points[inside_hull]

def plot_points_in_hull(azim=45, elev=30):
    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d', facecolor='whitesmoke')

    # Отображение точек внутри выпуклой оболочки
    ax.scatter(points_in_hull[:, 0], points_in_hull[:, 1], points_in_hull[:, 2],
               color='blue', s=1, alpha=0.3, label='Сетка точек')

    # Отображение атомов в виде сфер с учетом ван-дер-ваальсовых радиусов
    for atom, coord in zip(atoms, coords):
        color = atom_colors.get(atom, '#808080')  # Серый цвет по умолчанию
        radius = vdw_radii.get(atom, 1.5)  # Используем 1.5 Å по умолчанию
        # Создаем сферу
        u, v = np.mgrid[0:2*np.pi:30j, 0:np.pi:15j]
        x = radius * np.cos(u) * np.sin(v) + coord[0]
        y = radius * np.sin(u) * np.sin(v) + coord[1]
        z = radius * np.cos(v) + coord[2]
        ax.plot_surface(x, y, z, color=color, linewidth=0, antialiased=True, shade=True, alpha=0.6)

    # Отображение выпуклой оболочки
    for simplex in hull.simplices:
        triangle = coords[simplex]
        x_triangle = triangle[:, 0]
        y_triangle = triangle[:, 1]
        z_triangle = triangle[:, 2]
        ax.plot_trisurf(x_triangle, y_triangle, z_triangle, color='cyan', alpha=0, linewidth=0)

    # Настройка осей
    ax.set_xlabel('X координата', fontsize=14, fontweight='bold', labelpad=15)
    ax.set_ylabel('Y координата', fontsize=14, fontweight='bold', labelpad=15)
    ax.set_zlabel('Z координата', fontsize=14, fontweight='bold', labelpad=15)
    ax.set_title('3D визуализация молекулы с точками внутри выпуклой оболочки', fontsize=18, fontweight='bold', pad=30)

    # Настройка лимитов осей
    ax.set_xlim([min_coords[0] - 2, max_coords[0] + 2])
    ax.set_ylim([min_coords[1] - 2, max_coords[1] + 2])
    ax.set_zlim([min_coords[2] - 2, max_coords[2] + 2])

    ax.set_axis_off()

    # Установка углов обзора
    ax.view_init(elev=elev, azim=azim)

    # Удаление дубликатов в легенде
    unique_atoms = set(atoms)
    legend_elements = []
    for element in unique_atoms:
        color = atom_colors.get(element, '#808080')
        legend_elements.append(Line2D([0], [0], marker='o', color='w', label=element,
                                      markerfacecolor=color, markersize=14, markeredgecolor='k'))
    legend_elements.append(Line2D([0], [0], marker='o', color='w', label='Points in ConvexHull',
                                  markerfacecolor='blue', markersize=10, markeredgecolor='k'))

    # Добавление легенды
    ax.legend(handles=legend_elements, loc='upper right', fontsize=12, frameon=True, facecolor='white', edgecolor='black')

    # Настройка фона
    ax.grid(False)
    ax.xaxis.pane.set_edgecolor('w')
    ax.yaxis.pane.set_edgecolor('w')
    ax.zaxis.pane.set_edgecolor('w')

    # plt.savefig('step_5.png', format='png', dpi=300, bbox_inches='tight', pad_inches=0.1)

    # Показ графика
    plt.tight_layout()
    plt.show()

# Интерактивные слайдеры для управления углами
interact(plot_points_in_hull,
         azim=FloatSlider(min=0, max=360, step=1, value=360, description='Азимут:'),
         elev=FloatSlider(min=-90, max=90, step=1, value=-64, description='Угол высоты:'));

interactive(children=(FloatSlider(value=360.0, description='Азимут:', max=360.0, step=1.0), FloatSlider(value=…

# ШАГ 5. Поиск точек внутри атомов

In [46]:
grid_resolution = 0.3
atom_radii = np.array([vdw_radii.get(atom, 1.5) for atom in atoms])

# Вычисление выпуклой оболочки
hull = ConvexHull(coords)
hull_points = coords[hull.vertices]

# Определение границ сетки
max_radius = max(vdw_radii.values())
min_coords = np.min(hull_points, axis=0) - max_radius
max_coords = np.max(hull_points, axis=0) + max_radius

# Создание сетки точек
x = np.arange(min_coords[0], max_coords[0], grid_resolution)
y = np.arange(min_coords[1], max_coords[1], grid_resolution)
z = np.arange(min_coords[2], max_coords[2], grid_resolution)
X, Y, Z = np.meshgrid(x, y, z)
grid_points = np.vstack((X.ravel(), Y.ravel(), Z.ravel())).T

# Определение точек внутри выпуклой оболочки
delaunay = Delaunay(hull_points)
inside_hull = delaunay.find_simplex(grid_points) >= 0
points_in_hull = grid_points[inside_hull]

# Построение KD-дерева для эффективного поиска соседей
tree = cKDTree(coords)
max_atom_radius = max(atom_radii)
indices = tree.query_ball_point(points_in_hull, r=max_atom_radius)
inside_atom = np.zeros(len(points_in_hull), dtype=bool)

# Определение точек внутри атомов
for i, inds in enumerate(indices):
    point = points_in_hull[i]
    for j in inds:
        distance = np.linalg.norm(point - coords[j])
        if distance <= atom_radii[j]:
            inside_atom[i] = True
            break

def plot_molecule_with_points(azim=45, elev=30):
    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d', facecolor='whitesmoke')

    # Точки внутри атомов (зелёные)
    ax.scatter(points_in_hull[inside_atom, 0], points_in_hull[inside_atom, 1], points_in_hull[inside_atom, 2],
               color='forestgreen', s=4, alpha=0.2, label='Точки внутри атомов')

    # Точки в полостях (синие)
    ax.scatter(points_in_hull[~inside_atom, 0], points_in_hull[~inside_atom, 1], points_in_hull[~inside_atom, 2],
               color='blue', s=5, alpha=0.7, label='Точки в полостях')

    # Отображение атомов в виде сфер
    for atom, coord in zip(atoms, coords):
        color = atom_colors.get(atom, '#808080')  # Серый цвет по умолчанию
        radius = vdw_radii.get(atom, 1.5)
        # Создаем сферу для каждого атома
        u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
        x_sphere = radius * np.cos(u) * np.sin(v) + coord[0]
        y_sphere = radius * np.sin(u) * np.sin(v) + coord[1]
        z_sphere = radius * np.cos(v) + coord[2]
        ax.plot_surface(x_sphere, y_sphere, z_sphere, color=color, alpha=0.6)

    # Отображение выпуклой оболочки
    for simplex in hull.simplices:
        triangle = coords[simplex]
        x_triangle = triangle[:, 0]
        y_triangle = triangle[:, 1]
        z_triangle = triangle[:, 2]
        ax.plot_trisurf(x_triangle, y_triangle, z_triangle, color='cyan', alpha=0)

    # Настройка осей
    ax.set_title('3D визуализация молекулы с точками в полостях и внутри атомов', fontsize=18, fontweight='bold', pad=30)

    # Настройка лимитов осей
    ax.set_xlim([min_coords[0] - 2, max_coords[0] + 2])
    ax.set_ylim([min_coords[1] - 2, max_coords[1] + 2])
    ax.set_zlim([min_coords[2] - 2, max_coords[2] + 2])

    # Установка углов обзора
    ax.view_init(elev=elev, azim=azim)

    # Удаление дубликатов в легенде
    unique_atoms = set(atoms)
    legend_elements = []
    for element in unique_atoms:
        color = atom_colors.get(element, '#808080')
        legend_elements.append(Line2D([0], [0], marker='o', color='w', label=element,
                                      markerfacecolor=color, markersize=14, markeredgecolor='k'))
    legend_elements.append(Line2D([0], [0], marker='o', color='w', label='Points in the free cavities of the molecule',
                                  markerfacecolor='blue', markersize=10, markeredgecolor='k'))
    legend_elements.append(Line2D([0], [0], marker='o', color='w', label='Dots inside atoms',
                                  markerfacecolor='green', markersize=10, markeredgecolor='k'))

    # Добавление легенды
    ax.legend(handles=legend_elements, loc='upper right', fontsize=12, frameon=True, facecolor='white', edgecolor='black')

    # Настройка фона
    ax.grid(False)
    ax.set_axis_off()

    # plt.savefig('step_6.png', format='png', dpi=300, bbox_inches='tight', pad_inches=0.1)

    # Показ графика
    plt.tight_layout()
    plt.show()

# Интерактивные слайдеры для управления углами
interact(plot_molecule_with_points,
         azim=FloatSlider(min=0, max=360, step=1, value=360, description='Азимут:'),
         elev=FloatSlider(min=-90, max=90, step=1, value=-64, description='Угол высоты:'));

interactive(children=(FloatSlider(value=360.0, description='Азимут:', max=360.0, step=1.0), FloatSlider(value=…